In [1]:
import psycopg2

# Connect to an existing database
conn = psycopg2.connect("host=postgres port=5432 dbname=movies user=postgres password=changeme")

# Open a cursor to perform database operations
cur = conn.cursor()

# TASK 1
How many movies are in data set ?

In [2]:
cur.execute("SELECT COUNT(*) FROM movies;")
cur.fetchone()

(9742,)

Answer: (number of movies in dataset)

# TASK 2
What is the most common genre of movie?

In [ ]:
NOT DONE

# TASK 3
What are top 10 movies with highest rate ?

In [3]:
cur.execute(
    """SELECT "title", ROUND(AVG("rating"), 2)
    FROM movies
        JOIN ratings USING ("movieId")
    GROUP BY "movieId"
    ORDER BY AVG("rating") DESC
    LIMIT 10;"""
)
cur.fetchall()

[('Dream of Light (a.k.a. Quince Tree Sun, The) (Sol del membrillo, El) (1992)',
  Decimal('5.00')),
 ('I, the Jury (1982)', Decimal('5.00')),
 ('Palindromes (2004)', Decimal('5.00')),
 ('Hollywood Shuffle (1987)', Decimal('5.00')),
 ('7 Faces of Dr. Lao (1964)', Decimal('5.00')),
 ('Scooby-Doo! and the Loch Ness Monster (2004)', Decimal('5.00')),
 ('Nasu: Summer in Andalusia (2003)', Decimal('5.00')),
 ('Scooby-Doo! Abracadabra-Doo (2010)', Decimal('5.00')),
 ('20 Million Miles to Earth (1957)', Decimal('5.00')),
 ('Seve (2014)', Decimal('5.00'))]

Answer: (title, average rating)

# TASK 4 
What are 5 most often rating users ?

In [4]:
cur.execute(
    """SELECT "userId",
     COUNT("userId")
    FROM ratings
    GROUP BY  "userId"
    ORDER BY COUNT("userId") DESC
    LIMIT 10;"""
)
cur.fetchall()

[(414, 2698),
 (599, 2478),
 (474, 2108),
 (448, 1864),
 (274, 1346),
 (610, 1302),
 (68, 1260),
 (380, 1218),
 (606, 1115),
 (288, 1055)]

Answer(userID, number of ratings)

# TASK 5
When was done first and last rate included in data set and what was the rated movie tittle?

In [5]:
from datetime import datetime

cur.execute(
    """SELECT MIN("timestamp"), MAX("timestamp")
    FROM ratings"""
)
result = cur.fetchall()

oldest_rating = datetime.utcfromtimestamp(result[0][0]).strftime('%Y-%m-%d %H:%M:%S')
newest_rating = datetime.utcfromtimestamp(result[0][1]).strftime('%Y-%m-%d %H:%M:%S')

print(f"Oldest rating was created on {oldest_rating}")
print(f"Newest rating was created on {newest_rating}")


Oldest rating was created on 1996-03-29 18:36:55
Newest rating was created on 2018-09-24 14:27:30


# TASK 6
Find all movies released in 1990

In [14]:
cur.execute(
    "SELECT title FROM movies WHERE title LIKE '%(1990)%'"
)
result = cur.fetchall()
# print(result)
for i in result:
    print(i[0].removesuffix(' (1990)'))

Home Alone
Ghost
Dances with Wolves
Pretty Woman
Days of Thunder
Grifters, The
Tie Me Up! Tie Me Down! (¡Átame!)
Paris Is Burning
Goodfellas
Trust
Rosencrantz and Guildenstern Are Dead
Miller's Crossing
Femme Nikita, La (Nikita)
Pump Up the Volume
Cyrano de Bergerac
Amityville Curse, The
Die Hard 2
Young Guns II
Marked for Death
Hunt for Red October, The
King of New York
Metropolitan
Child's Play 2
Exorcist III, The
Gremlins 2: The New Batch
Back to the Future Part III
Godfather: Part III, The
Rescuers Down Under, The
NeverEnding Story II: The Next Chapter, The
My Blue Heaven
Sheltering Sky, The
Edward Scissorhands
Tales from the Darkside: The Movie
Heart Condition
Rocky V
Dick Tracy
Arachnophobia
Problem Child
Total Recall
Guardian, The
RoboCop 2
Bride of Re-Animator
Repossessed
Stanley & Iris
Awakenings
Come See the Paradise
Bonfire of the Vanities
Two Jakes, The
Pacific Heights
Mo' Better Blues
Teenage Mutant Ninja Turtles
Jacob's Ladder
Misery
All the Vermeers in New York
Flatliner

In [13]:
# Close communication with the database
cur.close()
conn.close()

Package                       Version
----------------------------- -----------
alembic                       1.7.7
altair                        4.2.0
anyio                         3.5.0
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
asttokens                     2.0.5
async-generator               1.10
attrs                         21.4.0
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
beautifulsoup4                4.11.1
bleach                        5.0.0
blinker                       1.4
bokeh                         2.4.2
Bottleneck                    1.3.4
brotlipy                      0.7.0
cached-property               1.5.2
certifi                       2021.10.8
certipy                       0.1.3
cffi                          1.15.0
charset-normalizer            2.0.12
click                         8.1.2
cloudpickle                   2.0.0
colorama                      0.4.4
conda        